In [27]:
#!pip install selenium
#!pip install pandas
#!pip install openpyxl

Importamos el módulo **sys**

In [3]:
import sys

Definimos la ruta base de búsqueda de paquetes y módulos

In [4]:
sys.path.append('../../')

Importamos los módulos necesarios de la librería **Selenium** de Python

In [28]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

import time
import datetime
import random
import os

import pandas as pd

Importamos los módulos **Data** y **Ficheros** de nuestra librería

In [6]:
import curso.api.Data as Data
import curso.api.Ficheros as Files
import curso.api.Cadena as Cadena
import curso.api.Texto as Texto
import curso.api.Input as Input

Estructura del esquema del archivo '**provincias.data**'

In [7]:
SCHEMA_PROVINCIA = {}
SCHEMA_PROVINCIA['id'] = Input.to_int
SCHEMA_PROVINCIA['provincia'] = Input.to_str
SCHEMA_PROVINCIA['id_ccaa'] = Input.to_int
SCHEMA_PROVINCIA['nombre'] = Input.to_str
SCHEMA_PROVINCIA['formato'] = Input.to_int

Estructura del esquema del archivo de **poblaciones**

In [8]:
SCHEMA_POBLACION = {}
SCHEMA_POBLACION['id'] = Input.to_int
SCHEMA_POBLACION['id_ccaa'] = Input.to_int
SCHEMA_POBLACION['id_provincia'] = Input.to_int
SCHEMA_POBLACION['poblacion'] = Input.to_str
SCHEMA_POBLACION['nombre'] = Input.to_str
SCHEMA_POBLACION['url'] = Input.to_str

Estructura del esquema del archivo de **temperaturas**

In [9]:
SCHEMA_DATA = {}
SCHEMA_DATA['fecha'] = Input.to_date
SCHEMA_DATA['id_ccaa'] = Input.to_int
SCHEMA_DATA['id_provincia'] = Input.to_int
SCHEMA_DATA['id_poblacion'] = Input.to_int
SCHEMA_DATA['hora'] = Input.to_str
SCHEMA_DATA['temperatura'] = Input.to_float
SCHEMA_DATA['viento'] = Input.to_float

Ruta base de los datasets de poblaciones

In [10]:
RUTA_BASE_ELTIEMPO = 'c:/datasets/data/eltiempo/'
RUTA_DATASETS_POBLACIONES = RUTA_BASE_ELTIEMPO + 'poblaciones/'

Nombres de archivos de los datasets

In [11]:
FILE_DATA_PROVINCIAS = RUTA_BASE_ELTIEMPO + 'provincias.data'
FILE_DATA_POBLACIONES = RUTA_BASE_ELTIEMPO + 'poblaciones.data'
FILE_DATA_TEMPERATURAS = RUTA_BASE_ELTIEMPO + 'temperaturas.data'

Cargamos el dataset '**provincias.data**'

In [12]:
dataset_provincias = Data.data_record(FILE_DATA_PROVINCIAS, header=True, schema=SCHEMA_PROVINCIA)

Mostramos el contenido del dataset **dataset_provincias**

In [13]:
dataset_provincias

[{'id': 1,
  'provincia': 'Araba',
  'id_ccaa': 16,
  'nombre': 'alava',
  'formato': 0},
 {'id': 2,
  'provincia': 'Albacete',
  'id_ccaa': 8,
  'nombre': 'albacete',
  'formato': 1},
 {'id': 3,
  'provincia': 'Alacant',
  'id_ccaa': 10,
  'nombre': 'alicante',
  'formato': 1},
 {'id': 4,
  'provincia': 'Almería',
  'id_ccaa': 1,
  'nombre': 'almeria',
  'formato': 1},
 {'id': 5,
  'provincia': 'Ávila',
  'id_ccaa': 7,
  'nombre': 'avila',
  'formato': 1},
 {'id': 6,
  'provincia': 'Badajoz',
  'id_ccaa': 11,
  'nombre': 'badajoz',
  'formato': 1},
 {'id': 7,
  'provincia': 'Illes Balears',
  'id_ccaa': 4,
  'nombre': 'baleares',
  'formato': 0},
 {'id': 8,
  'provincia': 'Barcelona',
  'id_ccaa': 9,
  'nombre': 'barcelona',
  'formato': 1},
 {'id': 9,
  'provincia': 'Burgos',
  'id_ccaa': 7,
  'nombre': 'burgos',
  'formato': 1},
 {'id': 10,
  'provincia': 'Cáceres',
  'id_ccaa': 11,
  'nombre': 'caceres',
  'formato': 1},
 {'id': 11,
  'provincia': 'Cádiz',
  'id_ccaa': 1,
  'nombre

Dirección del sitio web para realizar Web Scraping

In [14]:
URL_SITE_EL_TIEMPO = 'https://www.eltiempo.es/'

Funciones básicas para el funcionamiento de la aplicación Web Scraping

In [15]:
# función que recibe una colección tipo diccionario, agrega un nuevo campo 'url' a la colección y 
# devuelve la colección modificada
def to_url_eltiempo(data):
    # https://www.eltiempo.es/cantabria - formato 0
    # https://www.eltiempo.es/en-provincia-huelva - formato 1
    # https://www.eltiempo.es/melilla.html - formato 2
    url = ''
    if data['formato'] == 1:
        url = 'en-provincia-' + data['nombre']
    elif data['formato'] == 2:
        url = data['nombre'] + '.html'
    else:
        url = data['nombre']
    data['url'] = URL_SITE_EL_TIEMPO + url
    return data

# función que recibe una colección de tipo diccionario y devuelve los valores de la colección como una cadena de texto
def dict_to_string(data):
    return ';'.join(str(value) for value in data.values())

# función que devuelve todas las poblaciones del sitio web 'eltiempo.es'
def extraer_poblaciones_eltiempo(url_site, web_provincias):
    driver = webdriver.Chrome()
    try:
        driver.get(url_site)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a._10qqh8uq'))).click()
        time.sleep(3)
        dominio = url_site.split('/')[2]
        id_poblacion = 0
        header = ';'.join(SCHEMA_POBLACION.keys())
        for provincia in web_provincias:
            if provincia['formato'] == 2:
                continue
            print('Extrayendo de => ' + provincia['url'])
            driver.get(provincia['url'])
            time.sleep(5)
            scroll_origin = ScrollOrigin.from_viewport(10, 10)
            ActionChains(driver).scroll_from_origin(scroll_origin, 0, 1200).perform()
            ActionChains(driver).scroll_from_origin(scroll_origin, 0, 100).perform()
            data_poblaciones = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'article.modules.m_ccaaprovince.m39')))
            links = [(str(tag.get_attribute('href')), tag.text) for tag in data_poblaciones.find_elements(By.TAG_NAME, 'a')]
            poblaciones = []
            for url, text in links:
                id_poblacion += 1
                poblacion = {}
                poblacion['id'] = id_poblacion
                poblacion['id_ccaa'] = provincia['id_ccaa']
                poblacion['id_provincia'] = provincia['id']
                poblacion['poblacion'] = text
                poblacion['nombre'] = url.split('/')[-1].split('.')[0]
                poblacion['url'] = url
                poblaciones.append(poblacion)
            poblaciones = map(dict_to_string, poblaciones)
            Files.save_file_list(RUTA_DATASETS_POBLACIONES + provincia['nombre'] + '.data', poblaciones, header=header)
    finally:
        driver.quit()

Creamos la colección '**web_provincias**' utilizando el dataset ***dataset_provincias***

In [16]:
web_provincias = list(map(to_url_eltiempo, dataset_provincias))

Mostramos el contenido de la colección '**web_provincias**'

In [17]:
for p in web_provincias:
    print(p)
#Files.save_file_list('c:/datasets/data/eltiempo/provincias.links', web_provincias)

{'id': 1, 'provincia': 'Araba', 'id_ccaa': 16, 'nombre': 'alava', 'formato': 0, 'url': 'https://www.eltiempo.es/alava'}
{'id': 2, 'provincia': 'Albacete', 'id_ccaa': 8, 'nombre': 'albacete', 'formato': 1, 'url': 'https://www.eltiempo.es/en-provincia-albacete'}
{'id': 3, 'provincia': 'Alacant', 'id_ccaa': 10, 'nombre': 'alicante', 'formato': 1, 'url': 'https://www.eltiempo.es/en-provincia-alicante'}
{'id': 4, 'provincia': 'Almería', 'id_ccaa': 1, 'nombre': 'almeria', 'formato': 1, 'url': 'https://www.eltiempo.es/en-provincia-almeria'}
{'id': 5, 'provincia': 'Ávila', 'id_ccaa': 7, 'nombre': 'avila', 'formato': 1, 'url': 'https://www.eltiempo.es/en-provincia-avila'}
{'id': 6, 'provincia': 'Badajoz', 'id_ccaa': 11, 'nombre': 'badajoz', 'formato': 1, 'url': 'https://www.eltiempo.es/en-provincia-badajoz'}
{'id': 7, 'provincia': 'Illes Balears', 'id_ccaa': 4, 'nombre': 'baleares', 'formato': 0, 'url': 'https://www.eltiempo.es/baleares'}
{'id': 8, 'provincia': 'Barcelona', 'id_ccaa': 9, 'nombr

Web Scraping para obtener todas las poblaciones del sitio web **eltiempo.es** y crear *datasets de poblaciones* con las urls

In [19]:
extraer_poblaciones_eltiempo(URL_SITE_EL_TIEMPO, web_provincias)

Extrayendo de => https://www.eltiempo.es/alava
Extrayendo de => https://www.eltiempo.es/en-provincia-albacete
Extrayendo de => https://www.eltiempo.es/en-provincia-alicante
Extrayendo de => https://www.eltiempo.es/en-provincia-almeria
Extrayendo de => https://www.eltiempo.es/en-provincia-avila
Extrayendo de => https://www.eltiempo.es/en-provincia-badajoz
Extrayendo de => https://www.eltiempo.es/baleares
Extrayendo de => https://www.eltiempo.es/en-provincia-barcelona
Extrayendo de => https://www.eltiempo.es/en-provincia-burgos
Extrayendo de => https://www.eltiempo.es/en-provincia-caceres
Extrayendo de => https://www.eltiempo.es/en-provincia-cadiz
Extrayendo de => https://www.eltiempo.es/en-provincia-castellon
Extrayendo de => https://www.eltiempo.es/en-provincia-ciudad-real
Extrayendo de => https://www.eltiempo.es/en-provincia-cordoba
Extrayendo de => https://www.eltiempo.es/a-coruna
Extrayendo de => https://www.eltiempo.es/en-provincia-cuenca
Extrayendo de => https://www.eltiempo.es/en

KeyboardInterrupt: 

Función que devuelve la unión de todos los datasets de poblaciones generados

In [22]:
def get_by_id(id, collection):
    return list(filter(lambda c: c['id'] == id, collection))[0]
    
def merge(directory):
    data = []
    for file in Files.dir_files(directory):
        data += Data.data_record(file, header=True, schema=SCHEMA_POBLACION)
    return data

Realizamos la unión de todos los datasets de poblaciones generados

In [ ]:
dataset_poblaciones = merge(RUTA_DATASETS_POBLACIONES)
print(len(dataset_poblaciones))

Guardamos el dataset '**dataset_poblaciones**' que contiene todas las poblaciones

In [ ]:
dataset_poblaciones = map(dict_to_string, dataset_poblaciones)
header = ';'.join(SCHEMA_POBLACION.keys())
Files.save_file_list(FILE_DATA_POBLACIONES, dataset_poblaciones, header=header)

Limpiamos el dataset '**dataset_poblaciones**' de urls repetidas

In [ ]:
def remove_duplicate(collection, field):
    value = ''
    data = []
    for item in collection:
        data_field = Texto.to_abc(item[field])
        if data_field == value:
            continue
        value = data_field
        data.append(item)
    return data

Cargamos de nuevo el dataset '**dataset_poblaciones**'

In [ ]:
dataset_poblaciones = Data.data_record(FILE_DATA_POBLACIONES, header=True, schema=SCHEMA_POBLACION)

Eliminamos los enlaces duplicados que puedan aparecer

In [ ]:
print(len(dataset_poblaciones))
dataset_poblaciones = remove_duplicate(dataset_poblaciones, 'nombre')
print(len(dataset_poblaciones))

Guardamos el dataset '**dataset_poblaciones**'

In [ ]:
header = ';'.join(SCHEMA_POBLACION.keys())
dataset_poblaciones = map(dict_to_string, dataset_poblaciones)
Files.save_file_list(FILE_DATA_POBLACIONES, dataset_poblaciones, header=header)

In [20]:
def temperaturas_diarias(driver, poblacion):
    temperaturas = []
    for li in driver.find_element(By.ID, 'meteograma').find_elements(By.TAG_NAME, 'ul')[1].find_elements(By.TAG_NAME, 'li'):
        datos = Cadena.trim(li.text).split(' ')
        data = {}
        data['fecha'] = datetime.date.today()
        data['id_ccaa'] = Input.to_int(poblacion['id_ccaa'])
        data['id_provincia'] = Input.to_int(poblacion['id_provincia'])
        data['id_poblacion'] = Input.to_int(poblacion['id'])
        data['hora'] = datos[0]
        data['temperatura'] = Input.to_float(datos[1][:-1])
        data['viento'] = Input.to_float(datos[5])
        temperaturas.append(data)
    return temperaturas

In [23]:
dataset_provincias = Data.data_record(FILE_DATA_PROVINCIAS, header=True, schema=SCHEMA_PROVINCIA)
dataset_poblaciones = Data.data_record(FILE_DATA_POBLACIONES, header=True, schema=SCHEMA_POBLACION)

id_provincia = 11
provincia = get_by_id(id_provincia, dataset_provincias)
driver = webdriver.Chrome()

try:
    driver.get(URL_SITE_EL_TIEMPO)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a._10qqh8uq'))).click()
    time.sleep(3)
    if not os.path.exists(FILE_DATA_TEMPERATURAS):
        with open(FILE_DATA_TEMPERATURAS, mode='w', encoding='utf-8') as file:
            file.write(';'.join(SCHEMA_DATA.keys()) + '\n')
    dataset = list(filter(lambda poblacion: poblacion['id_provincia'] == provincia['id'], dataset_poblaciones))
    for n, poblacion in enumerate(dataset):
        if n == 10:
            break
        print(f"{provincia['provincia']} [{n + 1}:{len(dataset)}] '{poblacion['poblacion']}' => {poblacion['url']}")
        driver.get(poblacion['url'] + '?v=por_hora')
        time.sleep(5)
        temperaturas = temperaturas_diarias(driver, poblacion)
        dataset_temperaturas = list(map(dict_to_string, temperaturas))
        Files.save_file_list(FILE_DATA_TEMPERATURAS, dataset_temperaturas, mode='a')
finally:
    driver.quit()

Cádiz [1:186] 'Albaladejo' => https://www.eltiempo.es/en-provincia-cadiz/albaladejo.html
Cádiz [2:186] 'Alberite' => https://www.eltiempo.es/en-provincia-cadiz/alberite.html
Cádiz [3:186] 'Alcaidesa' => https://www.eltiempo.es/alcaidesa.html
Cádiz [4:186] 'Alcalá de los Gazules' => https://www.eltiempo.es/alcala-de-los-gazules.html
Cádiz [5:186] 'Alcalá del Valle' => https://www.eltiempo.es/alcala-del-valle.html
Cádiz [6:186] 'Alcántara' => https://www.eltiempo.es/en-provincia-cadiz/alcantara.html
Cádiz [7:186] 'Algar' => https://www.eltiempo.es/algar.html
Cádiz [8:186] 'Algeciras' => https://www.eltiempo.es/algeciras.html
Cádiz [9:186] 'Algodonales' => https://www.eltiempo.es/algodonales.html
Cádiz [10:186] 'Alíjar' => https://www.eltiempo.es/alijar.html


In [24]:
dataset_temperaturas = Data.data_record(FILE_DATA_TEMPERATURAS, header=True, schema=SCHEMA_DATA)

In [25]:
dataset_temperaturas

[{'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '00:00',
  'temperatura': 19.0,
  'viento': 4.0},
 {'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '01:00',
  'temperatura': 18.0,
  'viento': 4.0},
 {'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '02:00',
  'temperatura': 18.0,
  'viento': 4.0},
 {'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '03:00',
  'temperatura': 17.0,
  'viento': 5.0},
 {'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '04:00',
  'temperatura': 16.0,
  'viento': 5.0},
 {'fecha': datetime.date(2024, 6, 3),
  'id_ccaa': 1,
  'id_provincia': 11,
  'id_poblacion': 5463,
  'hora': '05:00',
  'temperatura': 15.0,
  'viento': 5.0},
 {'fecha': datetime.date(2024, 6, 3),
  

Cargamos el dataset de temperaturas en un DataFrame de **pandas**

In [29]:
df = pd.DataFrame(dataset_temperaturas)
df

,fecha,id_ccaa,id_provincia,id_poblacion,hora,temperatura,viento
0,2024-06-03,1,11,5463,00:00,19.0,4.0
1,2024-06-03,1,11,5463,01:00,18.0,4.0
2,2024-06-03,1,11,5463,02:00,18.0,4.0
3,2024-06-03,1,11,5463,03:00,17.0,5.0
4,2024-06-03,1,11,5463,04:00,16.0,5.0
...,...,...,...,...,...,...,...
235,2024-06-03,1,11,5472,19:00,24.0,13.0
236,2024-06-03,1,11,5472,20:00,23.0,12.0
237,2024-06-03,1,11,5472,21:00,22.0,10.0
238,2024-06-03,1,11,5472,22:00,21.0,9.0


Guardamos el dataframe de pandas como un archivo **json**

In [30]:
df.to_json(FILE_DATA_TEMPERATURAS + '.json', date_format='iso', date_unit='s')

Creamos el dataframe pandas 'df_json' utilizando el archivo json creado anteriormente

In [31]:
df_json = pd.read_json(FILE_DATA_TEMPERATURAS + '.json')
df_json['fecha'] = pd.to_datetime(df_json['fecha'])
df_json

,fecha,id_ccaa,id_provincia,id_poblacion,hora,temperatura,viento
0,2024-06-03,1,11,5463,00:00,19,4
1,2024-06-03,1,11,5463,01:00,18,4
2,2024-06-03,1,11,5463,02:00,18,4
3,2024-06-03,1,11,5463,03:00,17,5
4,2024-06-03,1,11,5463,04:00,16,5
...,...,...,...,...,...,...,...
235,2024-06-03,1,11,5472,19:00,24,13
236,2024-06-03,1,11,5472,20:00,23,12
237,2024-06-03,1,11,5472,21:00,22,10
238,2024-06-03,1,11,5472,22:00,21,9


Guardamos el dataframe de pandas como un archivo **Excel**

In [32]:
df.to_excel(FILE_DATA_TEMPERATURAS + '.xlsx')

Creamos el dataframe pandas 'df_excel' utilizando el archivo Excel creado anteriormente

In [ ]:
df_excel = pd.read_excel(FILE_DATA_TEMPERATURAS + '.xlsx')
df_excel